# AutoML

PyCaret
* Main Site - https://pycaret.org/
* Docs - https://pycaret.readthedocs.io/en/latest/

## Table of Contents

* [Setup and Preprocessing](#setup)  
* [Compare Models](#compare)  
* [Create Model](#create)  
* [Tune Model](#tune)  
* [Evaluate Model](#evaluate)  
* [Finalize and Store Model](#finalize_and_store)  

## Imports and Global Settings

In [2]:
import sys
import mlflow
import pandas as pd
from sqlalchemy import create_engine
from pycaret import regression as py_reg
from pycaret import classification as py_cls

sys.path.append('../')
from passkeys import RDS_ENDPOINT, RDS_PASSWORD

# Pandas Settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5


mlflow.set_tracking_uri('file:/home/jeff/Documents/Data_Science_Projects/NBA_Betting/models/AutoML')

## Database Connection

In [3]:
username = 'postgres'
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = 'nba_betting'
port = '5432'

connection = create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

### Datasets

In [4]:
df = pd.read_sql_table('nba_model_ready', connection)

<a id='basic_data_overview'></a>

## Basic Data Overview

In [5]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9185 entries, 0 to 9184
Data columns (total 1219 columns):
 #     Column                                 Non-Null Count  Dtype  
---    ------                                 --------------  -----  
 0     game_id                                9185 non-null   object 
 1     CLS_TARGET_home_margin_GT_home_spread  9185 non-null   bool   
 2     REG_TARGET_actual_home_margin          9185 non-null   float64
 3     home_team_num                          9185 non-null   int64  
 4     away_team_num                          9185 non-null   int64  
 5     league_year_end                        9185 non-null   int64  
 6     fd_line_home                           0 non-null      float64
 7     dk_line_home                           0 non-null      float64
 8     covers_consenses_home                  0 non-null      float64
 9     home_spread                            9185 non-null   float64
 10    gp                                     9157 

In [6]:
df.head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consenses_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_vla,gp_vla_std,win_rank,win_vla,win_vla_std,loss_rank,loss_vla,loss_vla_std,w_pct_rank,w_pct_vla,w_pct_vla_std,mins_rank,mins_vla,mins_vla_std,pts_rank,pts_vla,pts_vla_std,fgm_rank,fgm_vla,fgm_vla_std,fga_rank,fga_vla,fga_vla_std,fg_pct_rank,fg_pct_vla,fg_pct_vla_std,fg3m_rank,fg3m_vla,fg3m_vla_std,fg3a_rank,fg3a_vla,fg3a_vla_std,fg3_pct_rank,fg3_pct_vla,fg3_pct_vla_std,ftm_rank,ftm_vla,ftm_vla_std,fta_rank,fta_vla,fta_vla_std,ft_pct_rank,ft_pct_vla,ft_pct_vla_std,oreb_rank,oreb_vla,oreb_vla_std,dreb_rank,dreb_vla,dreb_vla_std,reb_rank,reb_vla,reb_vla_std,ast_rank,ast_vla,ast_vla_std,tov_rank,tov_vla,tov_vla_std,stl_rank,stl_vla,stl_vla_std,blk_rank,blk_vla,blk_vla_std,blka_rank,blka_vla,blka_vla_std,pf_rank,pf_vla,pf_vla_std,pfd_rank,pfd_vla,pfd_vla_std,p_m_rank,p_m_vla,p_m_vla_std,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_vla_opp,gp_vla_std_opp,win_rank_opp,win_vla_opp,win_vla_std_opp,loss_rank_opp,loss_vla_opp,loss_vla_std_opp,w_pct_rank_opp,w_pct_vla_opp,w_pct_vla_std_opp,mins_rank_opp,mins_vla_opp,mins_vla_std_opp,pts_rank_opp,pts_vla_opp,pts_vla_std_opp,fgm_rank_opp,fgm_vla_opp,fgm_vla_std_opp,fga_rank_opp,fga_vla_opp,fga_vla_std_opp,fg_pct_rank_opp,fg_pct_vla_opp,fg_pct_vla_std_opp,fg3m_rank_opp,fg3m_vla_opp,fg3m_vla_std_opp,fg3a_rank_opp,fg3a_vla_opp,fg3a_vla_std_opp,fg3_pct_rank_opp,fg3_pct_vla_opp,fg3_pct_vla_std_opp,ftm_rank_opp,ftm_vla_opp,ftm_vla_std_opp,fta_rank_opp,fta_vla_opp,fta_vla_std_opp,ft_pct_rank_opp,ft_pct_vla_opp,ft_pct_vla_std_opp,oreb_rank_opp,oreb_vla_opp,oreb_vla_std_opp,dreb_rank_opp,dreb_vla_opp,dreb_vla_std_opp,reb_rank_opp,reb_vla_opp,reb_vla_std_opp,ast_rank_opp,ast_vla_opp,ast_vla_std_opp,tov_rank_opp,tov_vla_opp,tov_vla_std_opp,stl_rank_opp,stl_vla_opp,stl_vla_std_opp,blk_rank_opp,blk_vla_opp,blk_vla_std_opp,blka_rank_opp,blka_vla_opp,blka_vla_std_opp,pf_rank_opp,pf_vla_opp,pf_vla_std_opp,pfd_rank_opp,pfd_vla_opp,pfd_vla_std_opp,p_m_rank_opp,p_m_vla_opp,p_m_vla_std_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_vla,offrtg_vla_std,defrtg_rank,defrtg_vla,defrtg_vla_std,netrtg_rank,netrtg_vla,netrtg_vla_std,ast_pct_rank,ast_pct_vla,ast_pct_vla_std,ast_v_tov_rank,ast_v_tov_vla,ast_v_tov_vla_std,ast_ratio_rank,ast_ratio_vla,...,opp_fgm_c3,opp_fga_c3,opp_fg_pct_c3,opp_fgm_atb3,opp_fga_atb3,opp_fg_pct_atb3,opp_fgm_ra_rank,opp_fgm_ra_vla,opp_fgm_ra_vla_std,opp_fga_ra_rank,opp_fga_ra_vla,opp_fga_ra_vla_std,opp_fg_pct_ra_rank,opp_fg_pct_ra_vla,opp_fg_pct_ra_vla_std,opp_fgm_paint_rank,opp_fgm_paint_vla,opp_fgm_paint_vla_std,opp_fga_paint_rank,opp_fga_paint_vla,opp_fga_paint_vla_std,opp_fg_pct_paint_rank,opp_fg_pct_paint_vla,opp_fg_pct_paint_vla_std,opp_fgm_mr_rank,opp_fgm_mr_vla,opp_fgm_mr_vla_std,opp_fga_mr_rank,opp_fga_mr_vla,opp_fga_mr_vla_std,opp_fg_pct_mr_rank,opp_fg_pct_mr_vla,opp_fg_pct_mr_vla_std,opp_fgm_lc3_rank,opp_fgm_lc3_vla,opp_fgm_lc3_vla_std,opp_fga_lc3_rank,opp_fga_lc3_vla,opp_fga_lc3_vla_std,opp_fg_pct_lc3_rank,opp_fg_pct_lc3_vla,opp_fg_pct_lc3_vla_std,opp_fgm_rc3_rank,opp_fgm_rc3_vla,opp_fgm_rc3_vla_std,opp_fga_rc3_rank,opp_fga_rc3_vla,opp_fga_rc3_vla_std,opp_fg_pct_rc3_rank,opp_fg_pct_rc3_vla,opp_fg_pct_rc3_vla_std,opp_fgm_c3_rank,opp_fgm_c3_vla,opp_fgm_c3_vla_std,opp_fga_c3_rank,opp_fga_c3_vla,opp_fga_c3_vla_std,opp_fg_pct_c3_rank,opp_fg_pct_c3_vla,opp_fg_pct_c3_vla_std,opp_fgm_atb3_rank,opp_fgm_atb3_vla,opp_fgm_atb3_vla_std,opp_fga_atb3_rank,opp_fga_atb3_vla,opp_fga_atb3_vla_std,opp_fg_pct_atb3

## PyCaret - Regression

<a id=setup></a>

### Setup and Preprocessing

In [7]:
features_to_drop = ['game_id', 'CLS_TARGET_home_margin_GT_home_spread']
keep_features = ['REG_TARGET_actual_home_margin',
                 'home_team_num', 'away_team_num',
                 'league_year_end', 'home_spread']
features_to_use = [feature for feature in list(df) if feature[-3:] == 'vla'] + keep_features

In [8]:
model_ready_df = df.drop(columns=features_to_drop)
model_ready_df = model_ready_df[features_to_use]

In [9]:
model_ready_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9185 entries, 0 to 9184
Data columns (total 156 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    gp_vla                         9157 non-null   float64
 1    win_vla                        9157 non-null   float64
 2    loss_vla                       9157 non-null   float64
 3    w_pct_vla                      9157 non-null   float64
 4    mins_vla                       9157 non-null   float64
 5    pts_vla                        9157 non-null   float64
 6    fgm_vla                        9157 non-null   float64
 7    fga_vla                        9157 non-null   float64
 8    fg_pct_vla                     9157 non-null   float64
 9    fg3m_vla                       9157 non-null   float64
 10   fg3a_vla                       9157 non-null   float64
 11   fg3_pct_vla                    9157 non-null   float64
 12   ftm_vla                        9

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [10]:
setup_params = {'log_experiment': True,
                'log_profile': False,
                'log_plots': False,
                'experiment_name': 'NBA_Betting_REG_vla_only',
                'data': model_ready_df,
                'target': 'REG_TARGET_actual_home_margin',
                'train_size': 0.7,
                'preprocess': True,
                'normalize': False,        # zscore
                'transformation': False,   # yeo-johnson power transform to make data more Gaussian
                'remove_outliers': False,  # using SVD
                'remove_multicollinearity': False,
                'polynomial_features': False,
                'trigonometry_features': False,
                'feature_interaction': False,
                'feature_ratio': False,
                'feature_selection': False,
                'feature_selection_threshold': 0.8,
                'pca': False,
                'pca_components': 10,
                'numeric_features': [],
                'ignore_features': []
               }

In [11]:
nba_betting_regression = py_reg.setup(**setup_params)

,Description,Value
0,session_id,5646
1,Target,REG_TARGET_actual_home_margin
2,Original Data,"(9185, 156)"
3,Missing Values,True
4,Numeric Features,154
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(6429, 155)"


Traceback (most recent call last):
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 336, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 175, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/jeff/Documents/Data_Science_Projects/NBA_Betting/models/AutoML/mlruns/meta.yaml' does not exist.
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/pycaret/internal/tabular.py", line 1738, in setup
    mlflow.create_experiment(exp_name_log)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/tracking/fluent.py", line 868, in

INFO:logs:setup() succesfully completed......................................


<a id=compare></a>

### Compare Models

In [12]:
best_3_models = py_reg.compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,9.8587,160.8621,12.6749,0.2070,0.9881,1.1562,0.0300
en,Elastic Net,9.8675,160.9862,12.6797,0.2065,1.0002,1.1579,0.0290
br,Bayesian Ridge,9.8941,161.6584,12.7061,0.2032,1.0223,1.1639,0.1140
omp,Orthogonal Matching Pursuit,9.9337,162.7996,12.7500,0.1978,1.0178,1.1811,0.0270
huber,Huber Regressor,9.9396,163.1747,12.7638,0.1961,1.0255,1.1896,0.5010
gbr,Gradient Boosting Regressor,9.9965,164.0857,12.7996,0.1916,1.0986,1.1752,9.9200
ridge,Ridge Regression,10.0003,164.0894,12.8018,0.1911,1.0421,1.2045,0.0270
ada,AdaBoost Regressor,10.0159,165.3013,12.8484,0.1852,1.1390,1.1535,3.3020
lr,Linear Regression,10.1266,167.8908,12.9496,0.1722,1.0505,1.2376,0.3370
lightgbm,Light Gradient Boosting Machine,10.2997,173.8499,13.1750,0.1434,1.0979,1.2535,1.0250


INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:display_container: 2
INFO:logs:[Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=5646,
      selection='cyclic', tol=0.0001, warm_start=False), ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=5646, selection='cyclic', tol=0.0001, warm_start=False), BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)]
INFO:logs:compare_models() succesfully completed......................................


<a id=create></a>

### Create Selected Model

In [13]:
model = py_reg.create_model('lasso')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,10.0672,169.2763,13.0106,0.2077,1.0361,1.1254
1,9.3162,145.2343,12.0513,0.2118,0.9266,1.2055
2,10.3527,171.2627,13.0867,0.1916,1.0526,1.1170
3,9.4964,145.7864,12.0742,0.1897,0.9538,1.1955
4,9.3520,144.5225,12.0218,0.2169,1.0163,1.0961
5,10.1951,169.3225,13.0124,0.1958,1.0069,1.2296
6,10.1891,174.5486,13.2117,0.2294,0.9854,1.0929
7,10.1042,169.2935,13.0113,0.1867,0.9395,1.1560
8,9.6919,153.5526,12.3916,0.2272,0.9960,1.1711
9,9.8219,165.8216,12.8772,0.2136,0.9680,1.1730


INFO:logs:create_model_container: 21
INFO:logs:master_model_container: 21
INFO:logs:display_container: 3
INFO:logs:Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=5646,
      selection='cyclic', tol=0.0001, warm_start=False)
INFO:logs:create_model() succesfully completed......................................


<a id=tune></a>

### Tune Selected Model

In [14]:
tuned_model = py_reg.tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,10.0573,169.1652,13.0064,0.2082,1.0361,1.1147
1,9.3100,144.9990,12.0416,0.2131,0.9278,1.1942
2,10.3512,171.2157,13.0849,0.1918,1.0471,1.1108
3,9.4762,145.4758,12.0613,0.1914,0.9578,1.1868
4,9.3491,144.2728,12.0114,0.2183,1.0133,1.0910
5,10.1949,169.4024,13.0155,0.1955,1.0068,1.2240
6,10.1821,174.4412,13.2076,0.2299,0.9879,1.0840
7,10.1191,169.3231,13.0124,0.1866,0.9437,1.1472
8,9.6962,153.8047,12.4018,0.2259,0.9996,1.1612
9,9.8007,165.2569,12.8552,0.2163,0.9747,1.1581


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 4
INFO:logs:Lasso(alpha=1.82, copy_X=True, fit_intercept=False, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=5646,
      selection='cyclic', tol=0.0001, warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


<a id=evaluate></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [15]:
py_reg.evaluate_model(tuned_model)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=Lasso(alpha=1.82, copy_X=True, fit_intercept=False, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=5646,
      selection='cyclic', tol=0.0001, warm_start=False), fold=None, fit_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [16]:
# py_reg.interpret_model(tuned_model)

<a id=finalize_and_store></a>

### Model Finalization and Storage

In [17]:
final_model = py_reg.finalize_model(tuned_model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=Lasso(alpha=1.82, copy_X=True, fit_intercept=False, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=5646,
      selection='cyclic', tol=0.0001, warm_start=False), fit_kwargs=None, groups=None, model_only=True, display=None)
INFO:logs:Finalizing Lasso(alpha=1.82, copy_X=True, fit_intercept=False, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=5646,
      selection='cyclic', tol=0.0001, warm_start=False)
INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=Lasso(alpha=1.82, copy_X=True, fit_intercept=False, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=5646,
      selection='cyclic', tol=0.0001, warm_start=False), fold=None, round=4, cross_validation=True, predict=True, fit_kwargs={}, groups=None, refit=True, verbose=False, system=False, metrics=None, add_to_model_list=False, probability

In [33]:
# py_reg.save_model(final_model, '../models/AutoML/Baseline_Lasso_Reg_PyCaret')

In [19]:
# !mlflow ui

## Classification

<a id=setup></a>

### Setup and Preprocessing

In [20]:
features_to_drop = ['game_id', 'REG_TARGET_actual_home_margin']
keep_features = ['CLS_TARGET_home_margin_GT_home_spread',
                 'home_team_num', 'away_team_num',
                 'league_year_end', 'home_spread']
features_to_use = [feature for feature in list(df) if feature[-3:] == 'vla'] + keep_features

In [21]:
model_ready_df = df.drop(columns=features_to_drop)
model_ready_df = model_ready_df[features_to_use]

In [22]:
model_ready_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9185 entries, 0 to 9184
Data columns (total 156 columns):
 #    Column                                 Non-Null Count  Dtype  
---   ------                                 --------------  -----  
 0    gp_vla                                 9157 non-null   float64
 1    win_vla                                9157 non-null   float64
 2    loss_vla                               9157 non-null   float64
 3    w_pct_vla                              9157 non-null   float64
 4    mins_vla                               9157 non-null   float64
 5    pts_vla                                9157 non-null   float64
 6    fgm_vla                                9157 non-null   float64
 7    fga_vla                                9157 non-null   float64
 8    fg_pct_vla                             9157 non-null   float64
 9    fg3m_vla                               9157 non-null   float64
 10   fg3a_vla                               9157 non-null   flo

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [23]:
setup_params = {'log_experiment': True,
                'log_profile': False,
                'log_plots': False,
                'experiment_name': 'NBA_Betting_CLS_vla_only',
                'data': model_ready_df,
                'target': 'CLS_TARGET_home_margin_GT_home_spread',
                'train_size': 0.7,
                'preprocess': True,
                'normalize': False,        # zscore
                'transformation': False,   # yeo-johnson power transform to make data more Gaussian
                'remove_outliers': False,  # using SVD
                'remove_multicollinearity': False,
                'polynomial_features': False,
                'trigonometry_features': False,
                'feature_interaction': False,
                'feature_ratio': False,
                'feature_selection': False,
                'feature_selection_threshold': 0.8,
                'pca': False,
                'pca_components': 10,
                'numeric_features': ['league_year_end'],
                'ignore_features': []
               }

In [24]:
nba_betting_classification = py_cls.setup(**setup_params)

,Description,Value
0,session_id,4113
1,Target,CLS_TARGET_home_margin_GT_home_spread
2,Target Type,Binary
3,Label Encoded,"False: 0, True: 1"
4,Original Data,"(9185, 156)"
5,Missing Values,True
6,Numeric Features,155
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:Logging experiment in MLFlow
Traceback (most recent call last):
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 336, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 175, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/jeff/Documents/Data_Science_Projects/NBA_Betting/models/AutoML/mlruns/meta.yaml' does not exist.
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/pycaret/internal/tabular.py", line 1738, in setup
    mlflow.create_experiment(exp_name_log)
  File "/home/jeff/anaconda3/lib/python3.7/site-packages/m

INFO:logs:setup() succesfully completed......................................


<a id=compare></a>

### Compare Models

In [25]:
best_3_models = py_cls.compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.5245,0.5188,0.4837,0.5160,0.4988,0.0476,0.0477,2.6650
xgboost,Extreme Gradient Boosting,0.5175,0.5230,0.4923,0.5081,0.4997,0.0341,0.0342,48.5310
lda,Linear Discriminant Analysis,0.5170,0.5201,0.4697,0.5088,0.4882,0.0324,0.0326,0.1460
catboost,CatBoost Classifier,0.5159,0.5185,0.4799,0.5073,0.4930,0.0306,0.0307,24.6000
et,Extra Trees Classifier,0.5142,0.5170,0.4808,0.5051,0.4923,0.0273,0.0273,1.0730
knn,K Neighbors Classifier,0.5141,0.5135,0.5074,0.5050,0.5061,0.0279,0.0279,0.2420
gbc,Gradient Boosting Classifier,0.5141,0.5159,0.4587,0.5055,0.4807,0.0262,0.0264,9.4130
lr,Logistic Regression,0.5131,0.5163,0.4526,0.5043,0.4768,0.0241,0.0243,1.0250
dt,Decision Tree Classifier,0.5127,0.5125,0.5017,0.5033,0.5024,0.0249,0.0250,0.5550
ridge,Ridge Classifier,0.5125,0.0000,0.4586,0.5038,0.4800,0.0232,0.0233,0.0270


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 2
INFO:logs:[RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=4113, verbose=0,
                       warm_start=False), XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_c

<a id=create></a>

### Create Selected Model

In [40]:
model = py_cls.create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.4914,0.4912,0.4114,0.4797,0.4429,-0.0198,-0.0200
1,0.4868,0.4773,0.4146,0.4746,0.4426,-0.0289,-0.0292
2,0.5226,0.5419,0.4557,0.5161,0.4840,0.0429,0.0432
3,0.5257,0.5235,0.4589,0.5197,0.4874,0.0492,0.0495
4,0.5179,0.5125,0.4620,0.5105,0.4850,0.0339,0.0341
5,0.4946,0.5031,0.4190,0.4818,0.4482,-0.0139,-0.0140
6,0.5241,0.5277,0.4825,0.5153,0.4984,0.0466,0.0467
7,0.5350,0.5435,0.4476,0.5301,0.4854,0.0667,0.0675
8,0.5179,0.5218,0.5016,0.5080,0.5048,0.0351,0.0351
9,0.5156,0.5201,0.4730,0.5068,0.4893,0.0296,0.0297


INFO:logs:create_model_container: 21
INFO:logs:master_model_container: 21
INFO:logs:display_container: 7
INFO:logs:LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=4113, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:create_model() succesfully completed......................................


<a id=tune></a>

### Tune Selected Model

In [41]:
tuned_model = py_cls.tune_model(model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.4977,0.4973,0.4810,0.4887,0.4848,-0.0052,-0.0052
1,0.4883,0.4815,0.4652,0.4788,0.4719,-0.0241,-0.0241
2,0.5288,0.5426,0.5190,0.5206,0.5198,0.0572,0.0572
3,0.5194,0.5240,0.5032,0.5113,0.5072,0.0383,0.0383
4,0.5194,0.5131,0.5063,0.5112,0.5087,0.0384,0.0384
5,0.4930,0.5057,0.4635,0.4818,0.4725,-0.0152,-0.0152
6,0.5210,0.5298,0.5429,0.5104,0.5262,0.0428,0.0429
7,0.5397,0.5416,0.5206,0.5307,0.5256,0.0786,0.0786
8,0.5163,0.5249,0.5429,0.5059,0.5237,0.0337,0.0337
9,0.5234,0.5204,0.5365,0.5137,0.5248,0.0472,0.0472


INFO:logs:create_model_container: 22
INFO:logs:master_model_container: 22
INFO:logs:display_container: 8
INFO:logs:LogisticRegression(C=5.038, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=4113, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


<a id=evaluate></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [42]:
py_cls.evaluate_model(tuned_model)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=LogisticRegression(C=5.038, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=4113, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), fold=None, fit_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [43]:
# py_cls.interpret_model(tuned_model)

<a id=finalize_and_store></a>

### Model Finalization and Storage

In [44]:
final_model = py_cls.finalize_model(tuned_model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=LogisticRegression(C=5.038, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=4113, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), fit_kwargs=None, groups=None, model_only=True, display=None)
INFO:logs:Finalizing LogisticRegression(C=5.038, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=4113, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=LogisticRegression(C=5.038, class_weight='balanced', dual=False,
                   fit_intercept=Tru

In [46]:
# py_cls.save_model(final_model, '../models/AutoML/Baseline_LR_CLS_PyCaret')

In [32]:
#!mlflow ui